<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# NeurIPS - Run ZSMMT

Goal : train models on ZSMMT

Data : ZSMMT

Models : SelfAttnLNP and ConvLNP and ConvCNP 

Loss : NLLLloss

Runs : 1

In [1]:
import os

os.chdir("..")

In [2]:
import submitit

from train_imgs import main,main2, parse_arguments
from results import get_exp_args

class Run:
    def checkpoint(self, args):
        """Resubmits the same callable with the same arguments but makes sure continnue from last chckpnt."""
        args.is_continue_train = True
        return submitit.utils.DelayedSubmission(self, args)

    def __call__(self, args):
        job_env = submitit.JobEnvironment()
        args.starting_run = args.starting_run * job_env.num_tasks + job_env.local_rank
        return main2(args)


In [3]:
args = get_exp_args("exp_mini", is_load=False)
len(args)

1

In [6]:
executor=submitit.AutoExecutor(folder="logs/%j", slurm_max_num_timeout=3)
executor.update_parameters(gpus_per_node=1, 
                           cpus_per_task=10, 
                           slurm_time=60*24*2, 
                           slurm_mem='32GB',
                           slurm_constraint="volta32gb",
                           slurm_partition="dev"
                          )

In [5]:
jobs = executor.map_array(Run(), args)

In [6]:
############################################################

In [7]:
jobs

[SlurmJob<job_id=28059968, task_id=0, state="RUNNING">]

In [9]:
for job in jobs:
    print("--------------------------------")
    print(job.stdout())

--------------------------------
submitit INFO (2020-07-09 14:14:20,569) - Starting with JobEnvironment(job_id=28059968, hostname=learnfair0338, local_rank=0(1), node=0(1), global_rank=0(1))
submitit INFO (2020-07-09 14:14:20,570) - Loading pickle: /private/home/yannd/projects/Neural-Process-Family/logs/28059968/28059968_submitted.pkl

--- Training mnist/ConvLNP/run_0 ---

  epoch    train_loss    valid_loss    cp        dur
-------  ------------  ------------  ----  ---------
      1    -1044.3515     -626.6722     +  1062.0232
Re-initializing module.
Re-initializing optimizer.
mnist/ConvLNP/run_0 | best epoch: 1 | train loss: -1044.3515 | valid loss: -626.6722 | test log likelihood: 548.9073
submitit INFO (2020-07-09 14:33:20,189) - Job completed successfully



In [9]:
for j in jobs:
    print("--------------------------------")
    print(j.stderr())

--------------------------------
None


In [14]:
for j in jobs:
    j.cancel()